In [1]:
import pickle
import keras as ks
import numpy as np
import tensorflow as tf
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras_contrib.layers import CRF
from keras_contrib.utils import save_load_utils

c:\users\katiohh\anaconda3\envs\ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VOCAB_SIZE = 4529
MAX_TEXT_LENGTH = 581


def ner_model():
    model = Sequential()
    model.add(
        Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=64,
            input_length=MAX_TEXT_LENGTH,
            trainable=False,
            mask_zero=True))
    model.add(
        Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.5)))
    model.add(
        Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.5)))
    model.add(
        Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.5)))
    model.add(Dense(7))
    crf = CRF(7)
    model.add(crf)
    model.compile(
        optimizer=ks.optimizers.Adadelta(),
        loss=crf.loss_function,
        metrics=[crf.accuracy])
    return model


def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)


def test_sentences(sentences, model, word_tokenizer, index_pos):
    vec = word_tokenizer.texts_to_sequences([sentences])
    vec = pad_sequences(vec, MAX_TEXT_LENGTH)
    ret = model.predict([vec])[0][-len(sentences):]
    ret = np.argmax(ret, axis=1) + 1
    return [index_pos[x] for x in ret]


def pretty_ner(sentences, ner_ret):
    for x, y in zip(sentences, ner_ret):
        print(x, y)


def init():
    model = ner_model()
    word_tokenizer = load_obj('model/word_tokenizer.pickle')
    index_pos = load_obj('model/index_pos.pickle')
    save_load_utils.load_all_weights(
        model, r'model/ner_stacked_lstm_weights.h5', include_optimizer=False)
    return model, word_tokenizer, index_pos


def split(line, n):
    return [line[i:i + n] for i in range(0, len(line), n)]


def output(sentences, ner_ret, width=20):
    ss = split(sentences, width)
    rs = split(ner_ret, width)
    for i, s in enumerate(ss):
        print('    '.join(ss[i]))
        for item in rs[i]:
            print('{:6}'.format(item), end='')
        print('\n')

In [11]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
ks.backend.set_session(sess)

In [3]:
with tf.device("/cpu:0"):
    _model, _word_tokenizer, _index_pos = init()

In [4]:
with tf.device("/cpu:0"):
    to_test = '近日，位于江苏的江苏精研股份有限公司（以下简称“公司”）收到全资子公司精研（东莞）科技发展有限公司（以下简称“东莞精研”）通知，东莞精研变更了其经营范围，履行了工商变更登记手续，并取得了东莞市工商行政管理局换发的《营业执照》。'
    ret = test_sentences(to_test, _model, _word_tokenizer, _index_pos)
    # pretty_ner(to_test, ret)
    output(to_test, ret)

近    日    ，    位    于    江    苏    的    江    苏    精    研    股    份    有    限    公    司    （    以
O     O     O     O     O     B-ORG I-ORG O     B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O     O     

下    简    称    “    公    司    ”    ）    收    到    全    资    子    公    司    精    研    （    东    莞
O     O     O     O     B-ORG I-ORG O     O     O     O     B-ORG I-ORG I-ORG I-ORG I-ORG O     O     O     B-ORG I-ORG 

）    科    技    发    展    有    限    公    司    （    以    下    简    称    “    东    莞    精    研    ”
I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG O     O     O     O     O     O     B-LOC I-LOC O     O     O     

）    通    知    ，    东    莞    精    研    变    更    了    其    经    营    范    围    ，    履    行    了
O     O     O     O     B-LOC I-LOC O     O     O     O     O     O     O     O     O     O     O     O     O     O     

工    商    变    更    登    记    手    续    ，    并    取    得    了    东    莞    市    工    商    行    政
O     O     O     O     O  